In [1]:
import numpy as np
from scipy.stats import norm

from lets_plot import *

In [2]:
LetsPlot.setup_html()

In [3]:
from lets_plot.plot.geom import _geom

_fun_x_name, _fun_y_name = 'x', 'y'

def _get_default_xrange(xlim, n):
    default_xlim = [0.0, 1.0]
    default_size = 512

    start, stop = xlim if xlim is not None else default_xlim
    size = n if n is not None else default_size

    return np.linspace(start, stop, size)

def _get_xrange(mapping, data, xlim, n):
    if mapping is None or 'x' not in mapping.as_dict():
        return _get_default_xrange(xlim, n)

    aes_x_value = mapping.as_dict()['x']

    if isinstance(aes_x_value, str):
        if data is None:
            return _get_default_xrange(xlim, n)
        return np.array(data[aes_x_value])

    if hasattr(aes_x_value, '__iter__'):
        return np.array(aes_x_value)

    raise Exception("Unknown type of mapped value: {0}".format(type(aes_x_value)))

def _get_fun_data(mapping, data, stat, fun, xlim, n):
    if fun is None:
        return {_fun_x_name: [], _fun_y_name: []}

    xs = _get_xrange(mapping, data, xlim, n)
    ys = np.vectorize(fun)(xs)

    return {_fun_x_name: xs, _fun_y_name: ys}

def _get_mapping(mapping, stat):
    mapping_dict = mapping.as_dict() if mapping is not None else {}
    fun_mapping_dict = {'x': _fun_x_name, 'y': _fun_y_name}

    return aes(**{**mapping_dict, **fun_mapping_dict})

def geom_function(mapping=None, *, data=None, stat=None, geom=None, position=None, show_legend=None, tooltips=None,
                  fun=None, xlim=None, n=None,
                  color_by=None,
                  **other_args):
    fun_stat = stat if stat is not None else 'identity'
    fun_geom = geom if geom is not None else 'line'
    return _geom(fun_geom,
                 mapping=_get_mapping(mapping, fun_stat),
                 data=_get_fun_data(mapping, data, fun_stat, fun, xlim, n),
                 stat=fun_stat,
                 position=position,
                 show_legend=show_legend,
                 sampling=None,
                 tooltips=tooltips,
                 color_by=color_by,
                 **other_args)

In [4]:
np.random.seed(42)

In [5]:
ggplot() + \
    geom_density(aes(x=np.random.normal(size=500))) + \
    geom_function(fun=norm.pdf, xlim=[-4, 4], color="red") + \
    ggtitle("Standard use case")

In [6]:
ggplot() + geom_function() + ggtitle("Without parameters")

In [7]:
ggplot() + geom_function(fun=norm.pdf) + ggtitle("Parameter 'fun'")

In [8]:
ggplot() + geom_function(fun=norm.pdf, xlim=[-3, 3]) + ggtitle("Parameter 'xlim'")

In [9]:
ggplot() + geom_function(fun=norm.pdf, xlim=[-3, 3], n=10) + ggtitle("Parameter 'n'")

In [10]:
ggplot() + geom_function(aes(x=np.linspace(-3, 3, 7)), fun=norm.pdf) + ggtitle("Parameter 'mapping'")

In [11]:
ggplot() + geom_function(aes(x='v'), data={'v': np.linspace(-3, 3, 7)}, fun=norm.pdf) + ggtitle("Parameter 'data'")

In [12]:
ggplot() + geom_function(aes(x=[0, 1, 1, 1, 2, 2]), fun=lambda t: t, stat='count') + ggtitle("Parameter 'stat'")

In [13]:
ggplot() + geom_function(fun=norm.pdf, xlim=[-3, 3], n=30, geom='histogram') + ggtitle("Parameter 'geom'")

In [14]:
x = np.random.uniform(-5, 5, size=100)
fun = lambda t: 2**t - t
ggplot() + \
    geom_function(aes(x=x), fun=fun, stat='identity', geom='point') + \
    geom_function(aes(x=x), fun=fun, stat='smooth', geom='smooth', deg=3) + \
    ggtitle("Parameters 'stat' and 'geom'")

In [15]:
ggplot() + \
    geom_function(fun=norm.pdf, xlim=[-3, 3], position=position_nudge(y=.01), color="red") + \
    geom_function(fun=norm.pdf, xlim=[-3, 3], position=position_nudge(y=-.01), color="blue") + \
    ggtitle("Paramter 'position'")

In [16]:
ggplot() + \
    geom_function(aes(color="y"), fun=norm.pdf, xlim=[-3, 3], show_legend=False) + \
    ggtitle("Parameter 'show_legend'")

In [17]:
ggplot() + \
    geom_function(aes(size="y"), fun=norm.pdf, xlim=[-3, 3], tooltips=layer_tooltips().line("(^x, ^y)")) + \
    ggtitle("Parameter 'tooltips'")

In [18]:
ggplot() + \
    geom_function(aes(paint_a='y'), fun=np.sin, xlim=[-2*np.pi, 2*np.pi], color_by='paint_a', size=1) + \
    geom_function(aes(paint_b='y'), fun=np.cos, xlim=[-2*np.pi, 2*np.pi], color_by='paint_b', size=1) + \
    scale_gradient('paint_a', low="red", high="blue") + \
    scale_gradient('paint_b', low="green", high="yellow") + \
    ggtitle("Parameter 'color_by'")

In [19]:
ggplot() + \
    geom_function(fun=lambda t: t**2, xlim=[-5, 5]) + \
    scale_y_continuous(trans='sqrt') + \
    coord_flip() + \
    theme_minimal() + flavor_darcula() + \
    ggtitle("Additional specs")

In [20]:
def pwdf(t):
    if t < -1:
        return -t
    elif t > 1:
        return t
    else:
        return None

ggplot() + geom_function(fun=pwdf, xlim=[-3, 3]) + ggtitle("Piecewise-defined function")